In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [6]:
def read_df(fname):
    df = pd.read_parquet(fname)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    return df

In [21]:
df = read_df('data/yellow_tripdata_2022-01.parquet')

In [8]:
df.duration.std()

46.44530513776499

In [9]:
df[(df.duration >= 1) & (df.duration <= 60)].shape[0] / df.shape[0]

0.9827547930522406

In [33]:
categorical = ['PULocationID', 'DOLocationID']
target = ['duration']

In [35]:
target = df['duration']
df = df[categorical]# + target]
df

,PULocationID,DOLocationID
0,142,236
1,236,42
2,166,166
3,114,68
4,68,163
...,...,...
2463926,90,170
2463927,107,75
2463928,113,246
2463929,148,164


In [36]:
dicts = df.to_dict(orient='records')
dv = DictVectorizer()

X_train = dv.fit_transform(dicts)

In [37]:
X_train.shape

(2463931, 2)

In [38]:
lr = LinearRegression()
lr.fit(X_train, target.values)

LinearRegression()

In [39]:
y_pred = lr.predict(X_train)

In [40]:
mean_squared_error(target.values, y_pred, squared=False)

46.42314493428529

In [48]:
df_valid = read_df('data/yellow_tripdata_2022-02.parquet')

In [49]:
df_valid

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.00,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.00,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.50,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.00,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.50,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.00,0.5,0.00,0.00,0.3,17.67,NaN,NaN,16.000000
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.00,0.5,2.00,0.00,0.3,19.81,NaN,NaN,12.250000
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.00,0.5,3.81,0.00,0.3,21.49,NaN,NaN,15.333333
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.00,0.5,1.71,0.00,0.3,17.54,NaN,NaN,8.266667


In [50]:
valid_target = df_valid['duration']
df_valid = df_valid[categorical]

In [51]:
val_dicts = df_valid.to_dict(orient='records')
X_valid = dv.transform(val_dicts)

In [52]:
y_pred = lr.predict(X_valid)
mean_squared_error(valid_target.values, y_pred, squared=False)

47.26263921133818